# Multi Layer Perceptron Model

- BOOK: [Predict the Future with MLPs, CNNs and LSTMs in Python](https://machinelearningmastery.com/deep-learning-for-time-series-forecasting/) by Jason Brownlee

In [1]:
# !pip3 uninstall -y torch torchvision
# !pip3 install torch torchvision

In [2]:
import os
import sys
import torch

import numpy as np
import pandas as pd

import torch.nn as nn
import torch.optim as optim

# Get the current working directory of the notebook
notebook_dir = os.getcwd()

# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../framework_for_time_series_data/tslearn/'))
from collections import namedtuple
from data_loader import build_stock_uts
from ml_models import MLP
from time_series import TimeSeriesFactory, UnivariateTimeSeries

In [3]:
observations = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [4]:
def split_sequence(sequence, n_steps_in, n_steps_out): 
    X, y = list(), list() 
    for i in range(len(sequence)): 
        # find the end of this pattern 
        end_ix = i + n_steps_in 
        out_end_ix = end_ix + n_steps_out 
        # check if we are beyond the sequence 
        if out_end_ix > len(sequence): 
            break
        # gather input and output parts of the pattern 
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix] 
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [5]:
X, y = split_sequence(observations, 3, 2)

In [6]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60],
       [50, 60, 70]])

In [7]:
y

array([[40, 50],
       [50, 60],
       [60, 70],
       [70, 80],
       [80, 90]])

In [8]:
def convert_uts_sequence_to_sml(uts_observations, prior_observations, forecasting_step):
    """Splits a given UTS into multiple input rows where each input row has a specified number of timestamps and the output is a single timestamp.
    
    Parameters:
    uts_observations -- 1D np array (of UTS data to transform to SML data with size  b rows/length x 1 dimension)
    prior_observations -- py int (of all observations before we get to where we want to start making the predictions)
    forecasting_step -- py int (of how far out to forecast, 1 only the next timestamp, 2 the next two timestamps, ... n the next n timestamps)
    
    Return:
    agg.values -- np array (of new sml data)
    """
    
    df = pd.DataFrame(uts_observations)
    cols = list()
    
    lag_col_names = []
    count_lag = 0
    # input sequence (t-n, ... t-1)
    for prior_observation in range(prior_observations, 0, -1):
        # print("prior_observation: ", prior_observation)
        cols.append(df.shift(prior_observation))
        new_col_name = "t-" + str(prior_observation)
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, forecasting_step):
        cols.append(df.shift(-i))
        new_col_name = "t"
        # print(new_col_name)
        lag_col_names.append(new_col_name)
        
        # put it all together
        uts_sml_df = pd.concat(cols, axis=1) 
        uts_sml_df.columns=[lag_col_names]
        # drop rows with NaN values
        uts_sml_df.dropna(inplace=True)
    
    # print(uts_sml_df)
    
    # colums to use to make prediction for last col
    X_train = uts_sml_df.iloc[:, 0: -1]
    
    # last column
    y_train = uts_sml_df.iloc[:, [-1]]
    return uts_sml_df

In [9]:
n_steps = 3
output_size = 2
converted_seq_df = convert_uts_sequence_to_sml(observations, n_steps, output_size)

In [10]:
converted_seq_df

,t-3,t-2,t-1,t,t
3,10.0,20.0,30.0,40,50.0
4,20.0,30.0,40.0,50,60.0
5,30.0,40.0,50.0,60,70.0
6,40.0,50.0,60.0,70,80.0
7,50.0,60.0,70.0,80,90.0


In [11]:
X = converted_seq_df.iloc[:, :n_steps]
X

,t-3,t-2,t-1
3,10.0,20.0,30.0
4,20.0,30.0,40.0
5,30.0,40.0,50.0
6,40.0,50.0,60.0
7,50.0,60.0,70.0


In [12]:
y = converted_seq_df.iloc[:, -output_size:]
y

,t,t
3,40,50.0
4,50,60.0
5,60,70.0
6,70,80.0
7,80,90.0


In [13]:
from keras.models import Sequential 
from keras.layers import Dense

model = Sequential() 
model.add(Dense(100, activation='relu' , input_dim=n_steps)) 
model.add(Dense(output_size)) 
model.compile(optimizer='adam' , loss='mse') 

2024-04-29 13:55:51.056732: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 13:55:55.733819: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# fit model 
model.fit(X, y, epochs=2000, verbose=0)
x_input = np.array([70, 80, 90])

In [15]:
# demonstrate prediction 
x_input = np.array([70, 80, 90]) 
x_input

array([70, 80, 90])

In [16]:
x_input_df = converted_seq_df.iloc[-1, -3:]
x_input_df

t-1    70.0
t      80.0
t      90.0
Name: 7, dtype: float64

In [17]:
x_input = np.array(x_input_df)
x_input = x_input.reshape((1, n_steps)) 
yhat = model.predict(x_input, verbose=0)

In [18]:
yhat

array([[100.07409, 109.98647]], dtype=float32)

In [19]:
hidden_size = 100

mlp_model = MLP(n_steps, hidden_size, output_size)
mlp_model

MLP(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=2, bias=True)
)

In [20]:
criterion = nn.MSELoss()
optimizer = optim.Adam(mlp_model.parameters())

# Example training loop
X = torch.randn(hidden_size, n_steps)  # Example input data
y = torch.randn(hidden_size, output_size)  # Example target data

epochs = 2000
for epoch in range(epochs):
    # Forward pass
    outputs = mlp_model(X)
    loss = criterion(outputs, y)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

Epoch [100/2000], Loss: 0.7246333360671997
Epoch [200/2000], Loss: 0.6682714223861694
Epoch [300/2000], Loss: 0.615033745765686
Epoch [400/2000], Loss: 0.563349723815918
Epoch [500/2000], Loss: 0.5165777802467346
Epoch [600/2000], Loss: 0.4755432903766632
Epoch [700/2000], Loss: 0.4371625483036041
Epoch [800/2000], Loss: 0.40146148204803467
Epoch [900/2000], Loss: 0.3696277141571045
Epoch [1000/2000], Loss: 0.33914652466773987
Epoch [1100/2000], Loss: 0.3093506097793579
Epoch [1200/2000], Loss: 0.2812107801437378
Epoch [1300/2000], Loss: 0.2551422715187073
Epoch [1400/2000], Loss: 0.2304910123348236
Epoch [1500/2000], Loss: 0.20590631663799286
Epoch [1600/2000], Loss: 0.18298910558223724
Epoch [1700/2000], Loss: 0.16336184740066528
Epoch [1800/2000], Loss: 0.14649046957492828
Epoch [1900/2000], Loss: 0.1313323676586151
Epoch [2000/2000], Loss: 0.115848109126091


In [21]:
x_input_df = converted_seq_df.iloc[-1, -3:]

In [22]:
x_input = torch.from_numpy(x_input).float()
x_input

tensor([[70., 80., 90.]])

In [23]:
x_input = x_input.reshape((1, n_steps)) 
x_input

tensor([[70., 80., 90.]])

In [25]:
# # Example prediction
# x_input_tensor = torch.tensor([[70, 80, 90]], dtype=torch.float32)  # Example input tensor
# yhat = mlp_model(x_input_tensor)  # Perform forward pass
# predicted_value = yhat.squeeze().item()  # Get the predicted value as a scalar
# print("Predicted Output:", predicted_value)

In [26]:
x_input_df = converted_seq_df.iloc[[-1], -3:].copy()
x_input_tensor = torch.tensor(x_input_df.values, dtype=torch.float32)
print("x_input_tensor shape:", x_input_tensor.shape)

x_input_tensor shape: torch.Size([1, 3])


In [27]:
# x_input_tensor = torch.tensor([[70, 80, 90]], dtype=torch.float32)  # Example input tensor
yhat = mlp_model(x_input_tensor)  # Perform forward pass

# Extract predicted values for each sample in the batch
predicted_values = yhat.squeeze(dim=1).tolist()  # Convert tensor to list of predicted values
print("Predicted Outputs:", predicted_values)

Predicted Outputs: [[149.30650329589844, 66.18651580810547]]
